In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

from tqdm import tqdm_notebook

In [2]:
home_twt= pd.read_csv(os.path.join('data','TWT', 'mapped', 'home_TWT.csv'), index_col=0, )

In [3]:
home_twt.head()

,user_id,home_ma
29,703776789,33044
1,12829832,43907
48,2723641674,33044
37,1529142049,29067
50,3511253537,33044


In [4]:
home_twt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 29 to 17
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  22 non-null     int64 
 1   home_ma  22 non-null     object
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


In [5]:
target_users= home_twt['user_id'].tolist()

In [6]:
import datetime
e_date = datetime.datetime.strptime('2022-03-05', '%Y-%m-%d')
i_date = datetime.datetime.strptime('2022-03-01', '%Y-%m-%d')

from datetime import date, timedelta

delta = e_date - i_date       # as timedelta

target_days = []
for i in range(delta.days + 1):
    day = i_date + timedelta(days=i)
    target_days.append(day)

min_ndays = len(target_days)

In [7]:
def read_tweets(target_days):
    twt_gdfs=[]
    for d in tqdm_notebook(target_days):
        day_for_file= d.strftime('%Y-%m-%d')

        try:
            twt_gdf= gpd.read_file(os.path.join("data","TWT","mapped", "map_point_tweets_{}.geojson".format(day_for_file)), 
                                   driver='GeoJSON', 
                                   encoding='utf-8',
                                  parse_dates=['timestamp'])
            twt_gdfs.append(twt_gdf)
        except:
            print("No file found for points: {}".format(day_for_file))

        try:
            twt_gdf= gpd.read_file(os.path.join("data", "TWT", "mapped", "map_poly_tweets_{}.geojson".format(day_for_file)), 
                                   driver='GeoJSON', 
                                   encoding='utf-8',
                                  parse_dates=['timestamp'])
            twt_gdfs.append(twt_gdf)
        except:
            print("No file found for polygon: {}".format(day_for_file))

    twt_gdf = pd.concat(twt_gdfs, ignore_index=True)
    twt_gdf['timestamp']= pd.to_datetime(twt_gdf['timestamp'])
    twt_gdf['date']= twt_gdf['timestamp'].apply(lambda x: x.date())
    return twt_gdf

In [ ]:
twt_gdf = read_tweets(target_days)

<ipython-input-7-8e90040a172a>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for d in tqdm_notebook(target_days):


  0%|          | 0/5 [00:00<?, ?it/s]

No file found for polygon: 2022-03-01


In [87]:
twt_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 45216 entries, 0 to 45215
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   tw_id        45216 non-null  int64              
 1   user_id      45216 non-null  int64              
 2   timestamp    45216 non-null  datetime64[ns, UTC]
 3   primary_key  45216 non-null  object             
 4   geometry     45216 non-null  geometry           
 5   place_name   36524 non-null  object             
 6   place_type   36524 non-null  object             
 7   Shape_area   36524 non-null  float64            
 8   date         45216 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), geometry(1), int64(2), object(4)
memory usage: 3.1+ MB


In [88]:
twt_gdf= twt_gdf[twt_gdf['user_id'].isin(target_users)]

In [89]:
twt_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 271 entries, 3363 to 45057
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   tw_id        271 non-null    int64              
 1   user_id      271 non-null    int64              
 2   timestamp    271 non-null    datetime64[ns, UTC]
 3   primary_key  271 non-null    object             
 4   geometry     271 non-null    geometry           
 5   place_name   269 non-null    object             
 6   place_type   269 non-null    object             
 7   Shape_area   269 non-null    float64            
 8   date         271 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(1), geometry(1), int64(2), object(4)
memory usage: 21.2+ KB


In [90]:
twt_gdf.head()

,tw_id,user_id,timestamp,primary_key,geometry,place_name,place_type,Shape_area,date
3363,1498760386719490052,703776789,2022-03-01 20:41:36+00:00,24092_AM,"POLYGON ((-5.82765 42.65650, -5.82544 42.65369...","Soto y Amío, España",city,0.019208,2022-03-01
3364,1498760949611646978,703776789,2022-03-01 20:43:50+00:00,24092_AM,"POLYGON ((-5.82765 42.65650, -5.82544 42.65369...","Soto y Amío, España",city,0.019208,2022-03-01
3421,1498679548849831937,289116150,2022-03-01 15:20:23+00:00,29901,"POLYGON ((-4.51500 36.60144, -4.52176 36.60362...",Tiki Restaurant Lounge Bar,poi,0.000000,2022-03-01
3458,1498762907689144329,802528374170275840,2022-03-01 20:51:37+00:00,3501903,"POLYGON ((-15.56806 27.74964, -15.56874 27.742...",Dunas de Maspalomas,poi,0.000000,2022-03-01
3501,1498780008034312193,417201261,2022-03-01 21:59:34+00:00,41029,"POLYGON ((-6.05955 37.37758, -6.06163 37.37727...",Hospital NISA Sevilla Aljarafe,poi,0.000000,2022-03-01


In [91]:
twt_w_home = twt_gdf.merge(home_twt, on='user_id')

In [92]:
home_twt[home_twt['user_id']==703776789]

,user_id,home_ma
29,703776789,33044


In [93]:
twt_w_home.head()

,tw_id,user_id,timestamp,primary_key,geometry,place_name,place_type,Shape_area,date,home_ma
0,1498760386719490052,703776789,2022-03-01 20:41:36+00:00,24092_AM,"POLYGON ((-5.82765 42.65650, -5.82544 42.65369...","Soto y Amío, España",city,0.019208,2022-03-01,33044
1,1498760949611646978,703776789,2022-03-01 20:43:50+00:00,24092_AM,"POLYGON ((-5.82765 42.65650, -5.82544 42.65369...","Soto y Amío, España",city,0.019208,2022-03-01,33044
2,1499040966514200584,703776789,2022-03-02 15:16:31+00:00,33044,"POLYGON ((-6.02010 43.27887, -6.02010 43.42743...","Oviedo, España",city,0.041270,2022-03-02,33044
3,1499425446840180736,703776789,2022-03-03 16:44:19+00:00,33044,"POLYGON ((-6.02010 43.27887, -6.02010 43.42743...","Oviedo, España",city,0.041270,2022-03-03,33044
4,1499528937411645440,703776789,2022-03-03 23:35:33+00:00,33044,"POLYGON ((-6.02010 43.27887, -6.02010 43.42743...","Oviedo, España",city,0.041270,2022-03-03,33044


In [ ]:
twt_w_home['timestamp']= twt_w_home['timestamp'].astype(str)
twt_w_home.to_file(os.path.join('data','TWT', 'twt_w_home.geojson'),driver='GeoJSON')

In [94]:
census_zones_gdf = gpd.read_file(os.path.join('data','agg_mun_tourist.geojson'),driver='GeoJSON')
census_zones_gdf.head()

,primary_key,area_km2,Viv-total,hotels-total,camps-total,POB20,HOMBRES,MUJERES,Viv-per-area,hotels-per-area,camps-per-area,geometry
0,01001_AM,359.212,17.0,4.0,1.0,7470,3945,3525,0.047326,0.011135,0.002784,"POLYGON ((-2.46644 42.78842, -2.46983 42.78803..."
1,01002,96.348,8.0,1.0,1.0,10264,5044,5220,0.083032,0.010379,0.010379,"MULTIPOLYGON (((-2.97525 42.93816, -2.97702 42..."
2,01010_AM,197.821,17.0,0.0,0.0,5971,3021,2950,0.085936,0.000000,0.000000,"POLYGON ((-3.00076 43.06800, -3.00853 43.06362..."
3,01031_AM,372.013,50.0,18.0,1.0,7428,3851,3577,0.134404,0.048385,0.002688,"MULTIPOLYGON (((-2.48257 42.49991, -2.48180 42..."
4,01036,37.559,5.0,1.0,0.0,18118,8868,9250,0.133124,0.026625,0.000000,"POLYGON ((-2.93850 43.13864, -2.93541 43.13658..."


In [95]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

def distance_between_ma(r1, r2):    
    lon1=  census_zones_gdf[census_zones_gdf['primary_key']==r1]['geometry'].values[0].centroid.x
    lat1= census_zones_gdf[census_zones_gdf['primary_key']==r1]['geometry'].values[0].centroid.y
    
    lon2=  census_zones_gdf[census_zones_gdf['primary_key']==r2]['geometry'].values[0].centroid.x
    lat2= census_zones_gdf[census_zones_gdf['primary_key']==r2]['geometry'].values[0].centroid.y
        
    d = haversine_np(lon1, lat1, lon2, lat2)
    return d

In [96]:
twt_w_home['dist']= twt_w_home.apply(lambda r: distance_between_ma(r['primary_key'], r['home_ma']), axis=1)

In [97]:
twt_w_home= twt_w_home[twt_w_home['dist']>=100]

In [98]:
twt_w_home.head()

,tw_id,user_id,timestamp,primary_key,geometry,place_name,place_type,Shape_area,date,home_ma,dist
48,1498757442163060736,452441985,2022-03-01 20:29:54+00:00,50159_AM,"POLYGON ((-1.50807 40.96065, -1.51204 40.95636...",Laguna de Gallocanta,poi,0.000000,2022-03-01,22907_AM,185.768049
82,1500128435858853899,1281529010711928833,2022-03-05 15:17:44+00:00,28079,"MULTIPOLYGON (((-3.88558 40.57446, -3.88522 40...",Txa Tei,poi,0.000000,2022-03-05,14021,303.350243
140,1500137701462495236,280665149,2022-03-05 15:54:33+00:00,12135,"POLYGON ((-0.17498 39.89170, -0.17498 39.98052...","Villarreal, España",city,0.010136,2022-03-05,08187,260.276082
141,1500172879287263241,280665149,2022-03-05 18:14:20+00:00,12135,"POLYGON ((-0.17498 39.89170, -0.17498 39.98052...","Villarreal, España",city,0.010136,2022-03-05,08187,260.276082
181,1500016975346638848,1364134795,2022-03-05 07:54:50+00:00,28131,"POLYGON ((-4.16943 40.60881, -4.16352 40.61388...",Monasterio de San Lorenzo de El Escorial,poi,0.000000,2022-03-05,29067,428.702788


In [107]:
n_user_per_ma= twt_w_home.groupby(['primary_key', 'date']).size()
n_user_per_ma= n_user_per_ma.reset_index()
n_user_per_ma.columns='ma date n_twt_visitors'.split()

In [108]:
n_user_per_ma.head()

,ma,date,n_twt_visitors
0,12135,2022-03-05,2
1,28005,2022-03-04,1
2,28079,2022-03-02,2
3,28079,2022-03-05,4
4,28131,2022-03-05,1


In [111]:
n_user_per_ma_ts= n_user_per_ma.pivot_table(index='date', columns='ma', values='n_twt_visitors').fillna(0)
n_user_per_ma_ts.head()

ma,12135,28005,28079,28131,33069_AM,39059,45168,50159_AM
date,,,,,,,,
2022-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2022-03-02,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0
2022-03-03,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2022-03-04,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2022-03-05,2.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0


In [112]:
n_user_per_ma_ts.to_csv(os.path.join('data', 'mobility', 'twt_tour_flows.csv'))